In [1]:
import pandas as pd
import json

# Join with Labels

In [2]:
df = pd.read_csv('../data/data.csv')
df['reads'] = df['reads'].map(lambda x: json.loads(x))
df['transcript_position'] = df['transcript_position'].astype(int)

In [3]:
df1 = pd.DataFrame(df['reads'].to_list())
df2 = pd.concat([df, df1], axis=1)
df2.head()

,transcript_id,transcript_position,fivemers,reads,0,1,2,3,4,5,6,7,8
0,ENST00000000233,244,AAGACCA,"[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0....",0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1
1,ENST00000000233,244,AAGACCA,"[0.0063100000000000005, 2.5300000000000002, 12...",0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9
2,ENST00000000233,244,AAGACCA,"[0.0046500000000000005, 3.92, 109.0, 0.0136000...",0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6
3,ENST00000000233,244,AAGACCA,"[0.00398, 2.06, 125.0, 0.0083, 5.01, 130.0, 0....",0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4
4,ENST00000000233,244,AAGACCA,"[0.006640000000000001, 2.92, 120.0, 0.00266, 3...",0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2


In [4]:
data = pd.read_csv('../data/data.info')

In [5]:
final_df = df2.merge(data, how='left', on=['transcript_id', 'transcript_position'])

In [6]:
final_df

,transcript_id,transcript_position,fivemers,reads,0,1,2,3,4,5,6,7,8,gene_id,label
0,ENST00000000233,244,AAGACCA,"[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0....",0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,"[0.0063100000000000005, 2.5300000000000002, 12...",0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,"[0.0046500000000000005, 3.92, 109.0, 0.0136000...",0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,"[0.00398, 2.06, 125.0, 0.0083, 5.01, 130.0, 0....",0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,"[0.006640000000000001, 2.92, 120.0, 0.00266, 3...",0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,"[0.0041800000000000006, 7.49, 108.0, 0.00564, ...",0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4,ENSG00000167747,0
11027102,ENST00000641834,1693,TTGACAT,"[0.006640000000000001, 1.9100000000000001, 109...",0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6,ENSG00000167747,0
11027103,ENST00000641834,1693,TTGACAT,"[0.00721, 4.58, 105.0, 0.00398, 6.58, 113.0, 0...",0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3,ENSG00000167747,0
11027104,ENST00000641834,1693,TTGACAT,"[0.00266, 2.33, 109.0, 0.009130000000000001, 1...",0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8,ENSG00000167747,0


In [7]:
final_df = final_df.rename(
    columns = {0:'dwelling_time_1', 1:'sd_current_1', 2:'mean_current_1',
               3:'dwelling_time_2', 4:'sd_current_2', 5:'mean_current_2',
               6:'dwelling_time_3', 7:'sd_current_3', 8:'mean_current_3'})
final_df

,transcript_id,transcript_position,fivemers,reads,dwelling_time_1,sd_current_1,mean_current_1,dwelling_time_2,sd_current_2,mean_current_2,dwelling_time_3,sd_current_3,mean_current_3,gene_id,label
0,ENST00000000233,244,AAGACCA,"[0.00299, 2.06, 125.0, 0.0177, 10.4, 122.0, 0....",0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,ENSG00000004059,0
1,ENST00000000233,244,AAGACCA,"[0.0063100000000000005, 2.5300000000000002, 12...",0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,ENSG00000004059,0
2,ENST00000000233,244,AAGACCA,"[0.0046500000000000005, 3.92, 109.0, 0.0136000...",0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,ENSG00000004059,0
3,ENST00000000233,244,AAGACCA,"[0.00398, 2.06, 125.0, 0.0083, 5.01, 130.0, 0....",0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,ENSG00000004059,0
4,ENST00000000233,244,AAGACCA,"[0.006640000000000001, 2.92, 120.0, 0.00266, 3...",0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,ENSG00000004059,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11027101,ENST00000641834,1693,TTGACAT,"[0.0041800000000000006, 7.49, 108.0, 0.00564, ...",0.00418,7.49,108.0,0.00564,10.20,116.0,0.01000,2.01,76.4,ENSG00000167747,0
11027102,ENST00000641834,1693,TTGACAT,"[0.006640000000000001, 1.9100000000000001, 109...",0.00664,1.91,109.0,0.00598,12.30,110.0,0.01760,2.61,74.6,ENSG00000167747,0
11027103,ENST00000641834,1693,TTGACAT,"[0.00721, 4.58, 105.0, 0.00398, 6.58, 113.0, 0...",0.00721,4.58,105.0,0.00398,6.58,113.0,0.00316,2.28,85.3,ENSG00000167747,0
11027104,ENST00000641834,1693,TTGACAT,"[0.00266, 2.33, 109.0, 0.009130000000000001, 1...",0.00266,2.33,109.0,0.00913,10.40,108.0,0.00664,4.44,76.8,ENSG00000167747,0


In [8]:
# aggregate and find min, max, mean, median, std, skew for each feature

lst = ['gene_id', 'transcript_id', 'transcript_position', 'fivemers', 'label']

grouped_df = final_df.groupby(by = lst).agg(
    {'dwelling_time_1': [min, max, 'mean', 'median', 'std', 'skew'],
     'sd_current_1': [min, max, 'mean', 'median', 'std', 'skew'],
     'mean_current_1': [min, max, 'mean', 'median', 'std', 'skew'],
     'dwelling_time_2': [min, max, 'mean', 'median', 'std', 'skew'],
     'sd_current_2': [min, max, 'mean', 'median', 'std', 'skew'],
     'mean_current_2': [min, max, 'mean', 'median', 'std', 'skew'],
     'dwelling_time_3': [min, max, 'mean', 'median', 'std', 'skew'],
     'sd_current_3': [min, max, 'mean', 'median', 'std', 'skew'],
     'mean_current_3': [min, max, 'mean', 'median', 'std', 'skew']
    }).reset_index()

In [9]:
grouped_df

gene_id    transcript_id transcript_position fivemers label  \
                                                                              
0       ENSG00000000003  ENST00000373020                 512  ATAACTC     0   
1       ENSG00000000003  ENST00000373020                 689  TAAACAA     0   
2       ENSG00000000003  ENST00000373020                 823  ATAACAA     0   
3       ENSG00000000003  ENST00000373020                 830  ATAACCA     0   
4       ENSG00000000003  ENST00000373020                 849  GTAACCC     0   
...                 ...              ...                 ...      ...   ...   
121833  ENSG00000284707  ENST00000641784                3122  CTGACCC     1   
121834  ENSG00000284707  ENST00000641784                3142  CTGACCC     0   
121835  ENSG00000284707  ENST00000641784                3224  GGGACTG     1   
121836  ENSG00000284707  ENST00000641784                3243  GGAACAA     0   
121837  ENSG00000284707  ENST00000641784                3266  CTAACTT     0   

       dwelling_time_1                                        ...  \
                   min     max      mean    median       std  ...   
0              0.00266  0.0169  0.007248  0.005990  0.004404  ...   
1              0.00232  0.0279  0.009868  0.007640  0.006946  ...   
2              0.00299  0.0196  0.007456  0.006310  0.003799  ...   
3              0.00266  0.0226  0.007765  0.006410  0.004869  ...   
4              0.00332  0.0181  0.006785  0.005980  0.003115  ...   
...                ...     ...       ...       ...       ...  ...   
121833         0.00232  0.0438  0.007506  0.005475  0.006832  ...   
121834         0.00232  0.0259  0.008712  0.007640  0.004966  ...   
121835         0.00232  0.0266  0.008452  0.006970  0.005276  ...   
121836         0.00204  0.0372  0.008700  0.007350  0.006147  ...   
121837         0.00232  0.0187  0.005972  0.005065  0.002963  ...   

       sd_current_3                            mean_current_3        \
               mean median       std      skew            min   max   
0          1.588750  1.570  0.366750 -0.081245           83.5  94.5   
1          2.198571  2.190  0.601085  1.726687           84.3  93.5   
2          2.306667  2.250  0.543519  0.417250           84.7  93.1   
3          2.020500  1.820  0.610146  0.759952           77.9  83.8   
4          1.924286  1.830  0.588367  1.625787           82.3  90.7   
...             ...    ...       ...       ...            ...   ...   
121833     2.844667  2.525  1.159143  0.861324           72.4  84.7   
121834     3.218689  3.040  1.406734  2.290015           71.2  83.5   
121835     3.443833  3.045  1.569948  1.131855           84.8  97.0   
121836     2.203750  2.130  0.688065  1.758215           80.3  97.4   
121837     1.977742  1.825  0.566480  1.518257           89.7  96.0   

                                              
             mean median       std      skew  
0       92.330000  92.55  2.258574 -3.353925  
1       89.385714  89.60  2.113122 -0.689415  
2       88.176190  88.20  2.420517  0.358926  
3       80.110000  80.00  1.523120  0.589444  
4       84.657143  84.40  2.048798  1.475521  
...           ...    ...       ...       ...  
121833  80.915000  81.15  2.405983 -1.034912  
121834  77.859016  78.00  2.407445 -0.367205  
121835  91.126667  91.45  2.901307 -0.080942  
121836  88.439063  88.70  3.335648 -0.153579  
121837  92.453226  92.65  1.313576  0.089409  

[121838 rows x 59 columns]

In [10]:
# rename the columns
grouped_df2 = grouped_df
grouped_df2.columns = ["_".join(x) for x in grouped_df.columns.ravel()]
grouped_df2 = grouped_df.rename(columns = {'gene_id_': 'gene_id', 
                                          'transcript_id_': 'transcript_id', 
                                          'transcript_position_': 'transcript_position',
                                          'fivemers_': 'sevenmers', 
                                          'label_': 'label'})

/var/folders/sg/256rs4d52q1fcb1g9mh332jc0000gn/T/ipykernel_28513/3491236702.py:3: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  grouped_df2.columns = ["_".join(x) for x in grouped_df.columns.ravel()]


In [11]:
grouped_df2

,gene_id,transcript_id,transcript_position,sevenmers,label,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_mean,dwelling_time_1_median,dwelling_time_1_std,...,sd_current_3_mean,sd_current_3_median,sd_current_3_std,sd_current_3_skew,mean_current_3_min,mean_current_3_max,mean_current_3_mean,mean_current_3_median,mean_current_3_std,mean_current_3_skew
0,ENSG00000000003,ENST00000373020,512,ATAACTC,0,0.00266,0.0169,0.007248,0.005990,0.004404,...,1.588750,1.570,0.366750,-0.081245,83.5,94.5,92.330000,92.55,2.258574,-3.353925
1,ENSG00000000003,ENST00000373020,689,TAAACAA,0,0.00232,0.0279,0.009868,0.007640,0.006946,...,2.198571,2.190,0.601085,1.726687,84.3,93.5,89.385714,89.60,2.113122,-0.689415
2,ENSG00000000003,ENST00000373020,823,ATAACAA,0,0.00299,0.0196,0.007456,0.006310,0.003799,...,2.306667,2.250,0.543519,0.417250,84.7,93.1,88.176190,88.20,2.420517,0.358926
3,ENSG00000000003,ENST00000373020,830,ATAACCA,0,0.00266,0.0226,0.007765,0.006410,0.004869,...,2.020500,1.820,0.610146,0.759952,77.9,83.8,80.110000,80.00,1.523120,0.589444
4,ENSG00000000003,ENST00000373020,849,GTAACCC,0,0.00332,0.0181,0.006785,0.005980,0.003115,...,1.924286,1.830,0.588367,1.625787,82.3,90.7,84.657143,84.40,2.048798,1.475521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENSG00000284707,ENST00000641784,3122,CTGACCC,1,0.00232,0.0438,0.007506,0.005475,0.006832,...,2.844667,2.525,1.159143,0.861324,72.4,84.7,80.915000,81.15,2.405983,-1.034912
121834,ENSG00000284707,ENST00000641784,3142,CTGACCC,0,0.00232,0.0259,0.008712,0.007640,0.004966,...,3.218689,3.040,1.406734,2.290015,71.2,83.5,77.859016,78.00,2.407445,-0.367205
121835,ENSG00000284707,ENST00000641784,3224,GGGACTG,1,0.00232,0.0266,0.008452,0.006970,0.005276,...,3.443833,3.045,1.569948,1.131855,84.8,97.0,91.126667,91.45,2.901307,-0.080942
121836,ENSG00000284707,ENST00000641784,3243,GGAACAA,0,0.00204,0.0372,0.008700,0.007350,0.006147,...,2.203750,2.130,0.688065,1.758215,80.3,97.4,88.439063,88.70,3.335648,-0.153579


In [12]:
pd.DataFrame(grouped_df2['sevenmers'].str.split('').to_list())[[x for x in range(1, 8)]]

,1,2,3,4,5,6,7
0,A,T,A,A,C,T,C
1,T,A,A,A,C,A,A
2,A,T,A,A,C,A,A
3,A,T,A,A,C,C,A
4,G,T,A,A,C,C,C
...,...,...,...,...,...,...,...
121833,C,T,G,A,C,C,C
121834,C,T,G,A,C,C,C
121835,G,G,G,A,C,T,G
121836,G,G,A,A,C,A,A


In [13]:
# split the sevenmers into seven columns
order_df = pd.DataFrame(grouped_df2['sevenmers'].str.split('').to_list())[[x for x in range(1, 8)]].rename(
    columns = {1: 'order_1', 2:'order_2', 
               3:'order_3', 4:'order_4', 
               5: 'order_5', 6:'order_6',
               7: 'order_7'})
grouped_df3 = pd.concat([grouped_df2, order_df], axis = 1)
grouped_df3

,gene_id,transcript_id,transcript_position,sevenmers,label,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_mean,dwelling_time_1_median,dwelling_time_1_std,...,mean_current_3_median,mean_current_3_std,mean_current_3_skew,order_1,order_2,order_3,order_4,order_5,order_6,order_7
0,ENSG00000000003,ENST00000373020,512,ATAACTC,0,0.00266,0.0169,0.007248,0.005990,0.004404,...,92.55,2.258574,-3.353925,A,T,A,A,C,T,C
1,ENSG00000000003,ENST00000373020,689,TAAACAA,0,0.00232,0.0279,0.009868,0.007640,0.006946,...,89.60,2.113122,-0.689415,T,A,A,A,C,A,A
2,ENSG00000000003,ENST00000373020,823,ATAACAA,0,0.00299,0.0196,0.007456,0.006310,0.003799,...,88.20,2.420517,0.358926,A,T,A,A,C,A,A
3,ENSG00000000003,ENST00000373020,830,ATAACCA,0,0.00266,0.0226,0.007765,0.006410,0.004869,...,80.00,1.523120,0.589444,A,T,A,A,C,C,A
4,ENSG00000000003,ENST00000373020,849,GTAACCC,0,0.00332,0.0181,0.006785,0.005980,0.003115,...,84.40,2.048798,1.475521,G,T,A,A,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENSG00000284707,ENST00000641784,3122,CTGACCC,1,0.00232,0.0438,0.007506,0.005475,0.006832,...,81.15,2.405983,-1.034912,C,T,G,A,C,C,C
121834,ENSG00000284707,ENST00000641784,3142,CTGACCC,0,0.00232,0.0259,0.008712,0.007640,0.004966,...,78.00,2.407445,-0.367205,C,T,G,A,C,C,C
121835,ENSG00000284707,ENST00000641784,3224,GGGACTG,1,0.00232,0.0266,0.008452,0.006970,0.005276,...,91.45,2.901307,-0.080942,G,G,G,A,C,T,G
121836,ENSG00000284707,ENST00000641784,3243,GGAACAA,0,0.00204,0.0372,0.008700,0.007350,0.006147,...,88.70,3.335648,-0.153579,G,G,A,A,C,A,A


In [14]:
# find the number of occurrence of a letter in a word
# input: str, str
# output: int
def find(word, letter):
    res = 0
    for i in word:
        if i==letter:
            res += 1
    return res

In [15]:
# count the A,C,G,T in the fivemers
grouped_df3['count_A'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'A'))
grouped_df3['count_C'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'C'))
grouped_df3['count_G'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'G'))
grouped_df3['count_T'] = grouped_df3['sevenmers'].map(lambda x: find(x, 'T'))
grouped_df3

,gene_id,transcript_id,transcript_position,sevenmers,label,dwelling_time_1_min,dwelling_time_1_max,dwelling_time_1_mean,dwelling_time_1_median,dwelling_time_1_std,...,order_2,order_3,order_4,order_5,order_6,order_7,count_A,count_C,count_G,count_T
0,ENSG00000000003,ENST00000373020,512,ATAACTC,0,0.00266,0.0169,0.007248,0.005990,0.004404,...,T,A,A,C,T,C,3,2,0,2
1,ENSG00000000003,ENST00000373020,689,TAAACAA,0,0.00232,0.0279,0.009868,0.007640,0.006946,...,A,A,A,C,A,A,5,1,0,1
2,ENSG00000000003,ENST00000373020,823,ATAACAA,0,0.00299,0.0196,0.007456,0.006310,0.003799,...,T,A,A,C,A,A,5,1,0,1
3,ENSG00000000003,ENST00000373020,830,ATAACCA,0,0.00266,0.0226,0.007765,0.006410,0.004869,...,T,A,A,C,C,A,4,2,0,1
4,ENSG00000000003,ENST00000373020,849,GTAACCC,0,0.00332,0.0181,0.006785,0.005980,0.003115,...,T,A,A,C,C,C,2,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENSG00000284707,ENST00000641784,3122,CTGACCC,1,0.00232,0.0438,0.007506,0.005475,0.006832,...,T,G,A,C,C,C,1,4,1,1
121834,ENSG00000284707,ENST00000641784,3142,CTGACCC,0,0.00232,0.0259,0.008712,0.007640,0.004966,...,T,G,A,C,C,C,1,4,1,1
121835,ENSG00000284707,ENST00000641784,3224,GGGACTG,1,0.00232,0.0266,0.008452,0.006970,0.005276,...,G,G,A,C,T,G,1,1,4,1
121836,ENSG00000284707,ENST00000641784,3243,GGAACAA,0,0.00204,0.0372,0.008700,0.007350,0.006147,...,G,A,A,C,A,A,4,1,2,0


# Export to csv

In [20]:
grouped_df3.to_csv('../data/grouped_data.csv', index=False)

In [21]:
small_grouped_df = grouped_df3.head(n=int(len(grouped_df3)/4))
small_grouped_df.to_csv('../data/small_grouped_data.csv', index=False) # a quarter of original